In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import linear_model 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.preprocessing import scale, StandardScaler, OneHotEncoder
import sklearn
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

In [20]:
df = pd.read_csv("data.csv")
df.head(2)

,esg_score21,esg_class21,environ_score21,environ_class21,social_score21,social_class21,govern_score21,govern_class21,turnover21,country,...,esg_exe_remuneration,esg_respect_humanr,esg_human_rights,esg_respect_freedom,esg_non_discrimination,esg_physical_risk,esg_energy_transition,esg_carbon_emission1,esg_carbon_emission2,esg_carbon_emission1_2
0,20.0000,Weak performance,0.0000,Weak performance,17.0000,Weak performance,36.0000,Limited performance,490490.0000,USA,...,28.0000,22.0000,25.0000,22.0000,27.0000,0.0000,0.0000,NaN,NaN,5537.0000
1,26.0000,Weak performance,18.0000,Weak performance,22.0000,Weak performance,40.0000,Limited performance,2122245.0000,USA,...,43.0000,31.0000,28.0000,23.0000,33.0000,43.0000,13.0000,2300.0000,40984.0000,43284.0000


In [24]:
def preprocess(df):
    """
    Clasifica columnas numéricas, realiza imputaciones de valores, aplica transformaciones específicas
    y guarda el DataFrame procesado en un archivo CSV.

    Args:
        df (pd.DataFrame): El DataFrame a procesar.

    Returns:
        pd.DataFrame: El DataFrame procesado.
    """
    # Clasificar columnas
    numericas = df.select_dtypes(include=['int64', 'float64']).columns
    binarias = [col for col in numericas if df[col].dropna().isin([0, 1]).all()]
    enteros = [col for col in numericas if col not in binarias and df[col].dtype == 'int64']
    decimales = [col for col in numericas if col not in binarias and df[col].dtype == 'float64']

    # Imputar NaNs con 0 en columnas binarias
    df[binarias] = df[binarias].fillna(0)

    # Definir las columnas específicas
    columnas_miles_usd = [
        'turnover21', 'ebt21', 'net_income21', 'cash_flow21', 'total_assets21', 'equity21',
        'gross_profit', 'no_current_ liabilities21', 'ono_current_liabilities21',
        'o_circu_liabilities21', 'liabilities21', 'inmo_inmat21', 'ebitda21'
    ]

    columnas_decimales = ['margen_ebitda21', 'debt_ratio21', 'esg_size21']

    # Seleccionar todas las columnas numéricas excepto las anteriores
    todas_columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    columnas_restantes = [col for col in todas_columnas_numericas 
                          if col not in columnas_miles_usd + columnas_decimales]

    # Aplicar transformaciones específicas

    # Convertir las columnas en miles de USD a millones
    for col in columnas_miles_usd:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir a numérico
            df[col] = (df[col] / 1_000).round(2)  # Convertir de miles a millones y redondear
            df[col] = df[col].fillna(df[col].median())  # Imputar mediana

    # Imputar mediana en columnas decimales
    for col in columnas_decimales:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir a numérico
            df[col] = df[col].fillna(df[col].median())  # Imputar mediana

    # Imputar mediana en el resto de columnas numéricas
    for col in columnas_restantes:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir a numérico
            df[col] = df[col].fillna(df[col].median())  # Imputar mediana

    # Imputar "Missing" para columnas categóricas específicas
    categorical_missing_cols = [
        'esg_class21', 'environ_class21', 'social_class21', 
        'govern_class21', 'activity', 'type_event', 'audit_status21'
    ]
    for col in categorical_missing_cols:
        if col in df.columns:
            df[col] = df[col].fillna("Missing")

    # Guardar el DataFrame procesado en un archivo CSV
    df.to_csv('data_2.csv', index=False)
    print("Archivo guardado como 'data_2.csv'")

    # Mostrar el número de valores NaN restantes
    n_nans = df.isna().sum().sum()
    print(f"Número total de valores NaN restantes: {n_nans}")

    return df

In [26]:
df = preprocess(df)

Archivo guardado como 'data_2.csv'
Número total de valores NaN restantes: 0
